<a href="https://colab.research.google.com/github/YahiaML/Manga-Downloader/blob/main/Manga_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fill in the info in the cell below

## Notes:
1. If you want to run the script on colab you have to set folder_path = r"/content".
2. If you want to download only 1 chapter, you have to set the chapter number in both start_from_chapter_num and stop_at_chapter_chapter_num.

In [1]:
# ----------  Insert the info  -----------#

# Insert the name of the manga/manhwa you want
Manga_name = "Example_Example"


# Insert one the chapters links of the manga/manhwa you want to download from any of these two sites:
# English: https://mangaonline.team/
# Arabic: https://azoranov.com/
url = "https://mangaonline.team/manga/solo-leveling/chapter-2/"


# Insert the folder path in which you want to save the downloaded mange/manhwa
#### PLEASE DON'T REMOVE THE r THAT IS FOUND BEFORE THE FOLDER PATH ####
folder_path = r"D:\folder_name\folder_name\folder_name" #### PLEASE DON'T REMOVE THE r THAT IS FOUND BEFORE THE FOLDER PATH ####

# You can use this version if you are going to run the script on colab
#folder_path = r"/content" #### PLEASE DON'T REMOVE THE r THAT IS FOUND BEFORE THE FOLDER PATH ####

# Please indicate the chapter number from which you would like to begin downloading.
start_from_chapter_num = 1

# Please indicate the chapter number at which you would like to stop downloading.
stop_at_chapter_chapter_num = 2

# Scraping script

## Press control + F9 to start

In [2]:
#Install required libraries
!pip install PyPDF2
!pip install natsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Import libraries
from bs4 import *
import requests
import os
from PIL import Image
from PIL import ImageFile
import os
from io import BytesIO
from PyPDF2 import PdfMerger 
from natsort import natsorted 
import shutil
import re

In [4]:
# ----------  Download chapters images  -----------#

urls, folders_names = [], []

for i in range(start_from_chapter_num, stop_at_chapter_chapter_num+1):
	urls.append(re.sub("\d+/$","{}/",url).format(i))
	folders_names.append(Manga_name + " {}".format(i))

for url,folder_name in zip(urls,folders_names):
		# CREATE FOLDER
		def folder_create(images,folder_name):
			try:
				# folder creation
				folder_name = folder_path.replace("\\", "/") + "/" +folder_name
				os.mkdir(folder_name)
		
			# if folder exists with that name, ask another name
			except:
				print("Folder Exist with that name!")
				folder_create()
		
			# image downloading start
			download_images(images, folder_name)
		
		
		# DOWNLOAD ALL IMAGES FROM THAT URL
		def download_images(images, folder_name):
		
			# initial count is zero
			count = 0
		
			# print total images found in URL
			print(f"Total {len(images)} Image Found!")
		
			# checking if images is not zero
			if len(images) != 0:
				for i, image in enumerate(images):
					# From image tag ,Fetch image Source URL
		
								# 1.data-srcset
								# 2.data-src
								# 3.data-fallback-src
								# 4.src
		
					# Here we will use exception handling
		
					# first we will search for "data-srcset" in img tag
					try:
						# In image tag ,searching for "data-srcset"
						image_link = image["data-srcset"]
						
					# then we will search for "data-src" in img
					# tag and so on..
					except:
						try:
							# In image tag ,searching for "data-src"
							image_link = image["data-src"]
						except:
							try:
								# In image tag ,searching for "data-fallback-src"
								image_link = image["data-fallback-src"]
							except:
								try:
									# In image tag ,searching for "src"
									image_link = image["src"]
		
								# if no Source URL found
								except:
									pass
								
					# After getting Image Source URL
					# We will try to get the content of image
					try:
						r = requests.get(image_link).content
						try:
						
							# possibility of decode
							r = str(r, 'utf-8')
		
						except UnicodeDecodeError:
						
							# After checking above condition, Image Download start
							with open(f"{folder_name}/images{i+1}.jpg", "wb+") as f:
								f.write(r)
		
							# counting number of image downloaded
							count += 1
					except:
						pass
					
				# There might be possible, that all
				# images not download
				# if all images download
				if count == len(images):
					print("All Images Downloaded!")
					
				# if all images not download
				else:
					print(f"Total {count} Images Downloaded Out of {len(images)}")
						
		# MAIN FUNCTION START
		def main(url):
		
			# content of URL
			r = requests.get(url)
		
			# Parse HTML Code
			soup = BeautifulSoup(r.text, 'html.parser')
		
			# find all images in URL
			images = soup.findAll('img')
		
			# Call folder create function
			folder_create(images,folder_name)
		
		# CALL MAIN FUNCTION
		main(url)

Total 50 Image Found!
All Images Downloaded!
Total 48 Image Found!
All Images Downloaded!


In [5]:
# ----------  Convert images to PDF  -----------#

download_dir = folder_path.replace("\\", "/")

for image_dir in natsorted([f for f in os.listdir(download_dir) if os.path.isdir(os.path.join(download_dir, f))]):
    pdf_merger = PdfMerger()

    for filename in natsorted(os.listdir(download_dir + "/" + image_dir)):       
        
        if filename.endswith('.png') or filename.endswith('.jpg'):
            with Image.open(os.path.join(download_dir + "/" + image_dir + "/" + filename)) as im:
                with BytesIO() as f:
                    ImageFile.LOAD_TRUNCATED_IMAGES = True
                    im.convert('RGB').save(f, format='pdf')
                    pdf_merger.append(BytesIO(f.getvalue()))   
             
    pdf_merger.write(os.path.join(download_dir, image_dir + ".pdf"))
    shutil.rmtree(os.path.join(download_dir, image_dir), ignore_errors=True)
    